In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-26 20:40:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2021-09-26 20:40:18 (1.22 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
#vine_df.show()

In [6]:
# Filter for >20 votes
filtered_vine_df = vine_df.filter('total_votes>20').select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
#filtered_vine_df.show()

In [7]:
#  Filter all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
add_vine_df = filtered_vine_df.withColumn('helpful_percent', filtered_vine_df['helpful_votes']/filtered_vine_df['total_votes'])
filter_greaterthan_50 = add_vine_df.filter('helpful_percent>.5').select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'helpful_percent'])
#filter_greaterthan_50.show()

In [8]:
# retrieve all the rows where a review was written as part of the Vine program (paid)
paid_vine_df = filter_greaterthan_50.filter(filter_greaterthan_50.vine == 'Y').select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'helpful_percent'])
#paid_vine_df.show()

In [9]:
# retrieve all the rows where a review was not written as part of the Vine program (unpaid)
unpaid_vine_df = filter_greaterthan_50.filter(filter_greaterthan_50.vine == 'N').select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'helpful_percent'])
#unpaid_vine_df.show()

In [10]:
# the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for Paid and Unpaid
total_paid = paid_vine_df.count()
print(f'Total Paid: {total_paid}')
five_star_paid = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
print(f'5 Star Paid: {five_star_paid}')
per_five_star_paid = round((five_star_paid / total_paid) * 100,2)
print(f'Percentage of 5 Star reviews for Paid: {per_five_star_paid}%')
print('------------------------------------------------')
total_unpaid = unpaid_vine_df.count()
print(f'Total Unpaid: {total_unpaid}')
five_star_unpaid = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
print(f'5 Star Unpaid: {five_star_unpaid}')
per_five_star_unpaid = round((five_star_unpaid / total_unpaid) * 100,2)
print(f'Percentage of 5 Star reviews for Unpaid: {per_five_star_unpaid}%')

Total Paid: 43
5 Star Paid: 14
Percentage of 5 Star reviews for Paid: 32.56%
------------------------------------------------
Total Unpaid: 7720
5 Star Unpaid: 4018
Percentage of 5 Star reviews for Unpaid: 52.05%
